<a href="https://colab.research.google.com/github/Samuel-Schlemper-Schlemuel/Sms_text_classification/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
df_train = pd.read_csv(train_file_path, delimiter='\t', names=['result', 'message'])
df_test = pd.read_csv(test_file_path, delimiter='\t', names=['result', 'message'])

In [4]:
df_train

,result,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [5]:
max_words = 10000
max_len = 100

In [6]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_train['message'])
sequences_train = tokenizer.texts_to_sequences(df_train['message'])
sequences_test = tokenizer.texts_to_sequences(df_test['message'])

In [7]:
x_train = pad_sequences(sequences_train, maxlen=max_len)
x_test = pad_sequences(sequences_test, maxlen=max_len)
y_train = pd.get_dummies(df_train['result']).values
y_test = pd.get_dummies(df_test['result']).values

In [8]:
model = Sequential()
model.add(Embedding(max_words, 100, input_length=max_len))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 conv1d (Conv1D)             (None, 96, 64)            32064     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1,036,354
Trainable params: 1,036,354
Non-trainable params: 0
______________________________________________

In [9]:
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
131/131 [==============================] - 27s 119ms/step - loss: 0.3241 - accuracy: 0.8899 - val_loss: 0.0717 - val_accuracy: 0.9849
Epoch 2/5
131/131 [==============================] - 7s 57ms/step - loss: 0.0323 - accuracy: 0.9911 - val_loss: 0.0453 - val_accuracy: 0.9871
Epoch 3/5
131/131 [==============================] - 3s 26ms/step - loss: 0.0062 - accuracy: 0.9988 - val_loss: 0.0455 - val_accuracy: 0.9878
Epoch 4/5
131/131 [==============================] - 4s 29ms/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.0491 - val_accuracy: 0.9885
Epoch 5/5
131/131 [==============================] - 4s 31ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0513 - val_accuracy: 0.9885


In [10]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  text = pd.DataFrame({'message': [pred_text]})
  sequence = tokenizer.texts_to_sequences(text['message'])
  predict = pad_sequences(sequence, maxlen=max_len)
  prediction = model.predict(predict)
  limiar = 0.5

  if prediction[0][0] > limiar:
      result = [prediction[0][0], 'ham']
  else:
      result =[prediction[0][1], 'spam']

  return result

pred_text = "how are you?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 141ms/step
[0.9999995, 'ham']


In [11]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 21ms/step
You passed the challenge. Great job!
